In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'kasey'

exp90_path = base_path/'2024.03.31_exp90'/'export'
exp90_2_path = base_path/'2024.04.02_exp90.2'/'export'
exp90_3_path = base_path/'2024.04.02_exp90.3'/'export'
exp90_4_path = base_path/'2024.04.05_exp90.4'/'export'
exp91_path = base_path/'2024.04.08_exp91'/'export'
exp92_path = base_path/'2024.04.12_exp92'/'export'

plates = pd.DataFrame({
    'data_path': [exp90_path/'plate1', exp90_path/'plate2', 
                  exp90_2_path, exp90_4_path,
                  exp90_3_path/'plate1', exp90_3_path/'plate2', 
                  exp91_path/'plate1.1', exp91_path/'plate1.2', exp91_path/'plate1.3', 
                  exp91_path/'plate2.1', exp91_path/'plate2.2', exp91_path/'plate2.3',
                  exp92_path/'plate1.1', exp92_path/'plate1.2', exp92_path/'plate1.3', 
                  exp92_path/'plate2.1', exp92_path/'plate2.2', exp92_path/'plate2.3',],
    
    'yaml_path': ([exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', 
                   exp90_path/'exp90_plate2_wells.yaml', exp90_path/'exp90_plate1_wells.yaml',
                   exp90_path/'exp90_plate1_wells.yaml', exp90_path/'exp90_plate2_wells.yaml', ] +
                  [exp91_path/'exp91_plate1_wells.yaml']*3 + 
                  [exp91_path/'exp91_plate2.1_wells.yaml', exp91_path/'exp91_plate2.2_wells.yaml', exp91_path/'exp91_plate2.3_wells.yaml'] +
                  [exp92_path/'exp92_plate1_wells.yaml', exp92_path/'exp92_plate1.2_wells.yaml', exp92_path/'exp92_plate1_wells.yaml',
                   exp92_path/'exp92_plate2_wells.yaml', exp92_path/'exp92_plate2.2_wells.yaml', exp92_path/'exp92_plate2_wells.yaml',]
                  ),
    
    'biorep': ([1, 1, 
                2, 2, 
                3, 3,] + 
                [1, 2, 3,]*4),
    
    'exp': (['exp90', 'exp90', 
             'exp90.2', 'exp90.4', 
             'exp90.3', 'exp90.3',] + 
            ['exp91']*6 + 
            ['exp92']*6)
})

output_path = rd.rootdir/'output'/'fig_tuning-supp'
cache_path = rd.rootdir/'output'/'fig_overview'/'data.gzip'

# Load data
data = pd.DataFrame()
if cache_path.is_file(): data = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data = data[data[c]>0]
    
    data.dropna(inplace=True)
    data.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata = base.get_metadata(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'construct-metadata.xlsx')
data = data.merge(metadata, how='left', on='construct')
display(data)


In [ ]:
# Create dicts to specify colors/markers
metadata['miR_'] = (metadata['miR'].isin(['none','na'])).map({True: '–', False:'+'})
metadata['ts_'] = (metadata['ts'].isin(['none','na'])).map({True: '–', False:'+'})
metadata_dict = metadata.set_index('construct').to_dict('dict')
construct_palette = metadata_dict['color'] # colored by design
construct_markers = metadata_dict['markers']

# Create unified color palette for most tuning
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='miR'), 'color'] = base.colors['purple']
metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR'].isin(['miR.FF5','miRE.FF5'])), 'color'] = metadata2.loc[(metadata2['group']=='miR') & (metadata2['miR'].isin(['miR.FF5','miRE.FF5'])), 'color'].apply(base.get_light_color)

metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'color'] = base.colors['blue']
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF6x1','FF6x2','FF6x4'])), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF6x1','FF6x2','FF6x4'])), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF4x1','FF4x2','FF4x4'])), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts'].isin(['FF4x1','FF4x2','FF4x4'])), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF3x1'), 'color'] = metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])) & (metadata2['ts']=='FF3x1'), 'color'].apply(base.get_light_color).apply(base.get_light_color)

metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miR.FF5'), 'color'] = base.colors['red']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miRE.FF5'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miR.FF4'), 'color'] = base.colors['green']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['miR']=='miRE.FF4'), 'color'] = base.colors['teal']

metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['miR'].isin(['miR.FF4','miRE.FF4'])), 'color'] = metadata2.loc[(metadata2['group']=='controller') & (metadata2['ts_kind']=='NT') & (metadata2['miR'].isin(['miR.FF4','miRE.FF4'])), 'color'].apply(base.get_light_color)

metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==2)), 'color'] = metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==2)), 'color'].apply(base.get_light_color)
metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==4)), 'color'] = metadata2.loc[((metadata2['group']=='controller') & (metadata2['ts_num']==4)), 'color'].apply(base.get_dark_color)

metadata2.loc[(metadata2['ts']=='FF6x3.4x1'), 'color'] = metadata2.loc[(metadata2['ts']=='FF6x3.4x1'), 'color'].apply(base.get_light_color)

# markers
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'markers'] = 'D'
metadata2.loc[(metadata2['miR'].isin(['miRE.FF5','miRE.FF4'])), 'markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'
metadata2.loc[(metadata2['group'].isin(['ts3','ts5'])), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'markers'] = 'D'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'markers'] = '^'
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'markers'] = 'v'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'
metadata2.loc[(metadata2['miR'].isin(['miR.FF5','miR.FF4'])), 'linestyle'] = '--'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
main_palette = metadata_dict2['color']
main_markers = metadata_dict2['markers']

In [ ]:
# Create color palette for comparing promoters
metadata2 = metadata.copy()
metadata2.loc[(metadata2['group']=='marker'), 'color'] = 'black'
metadata2.loc[(metadata2['group']=='base'), 'color'] = 'black'

metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EF1a'), 'color'] = base.colors['teal']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='CAG'), 'color'] = base.colors['orange']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='EFS'), 'color'] = base.colors['red']
metadata2.loc[(metadata2['group']=='controller') & (metadata2['promoter']=='hPGK'), 'color'] = base.colors['green']

metadata2.loc[(metadata2['ts_kind']=='NT'), 'color'] = base.colors['gray']
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EF1a'), 'color'].apply(base.get_light_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='EFS'), 'color'].apply(base.get_dark_color)
metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'] = metadata2.loc[(metadata2['ts_kind']=='NT') & (metadata2['promoter']=='hPGK'), 'color'].apply(base.get_dark_color).apply(base.get_dark_color)

# markers
metadata2['markers'] = 'o'
metadata2.loc[(metadata2['group']=='base'), 'markers'] = 'X'

# linestyles
metadata2['linestyle'] = '-' # default is solid
metadata2.loc[metadata2['group']=='marker', 'linestyle'] = ':'

metadata_dict2 = metadata2.set_index('construct').to_dict('dict')
promoter_palette = metadata_dict2['color']
promoter_markers = metadata_dict2['markers']

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['tagBFP-A', 'mGL-A', 'mCherry-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data[data['construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Add missing gates
gates.loc[len(gates.index)] = ['exp90.4',0,0,0,0,]  
gates.loc[gates['exp']=='exp90.4', channel_list] = gates.loc[gates['exp']=='exp90.2', channel_list].values

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mGL-A'
gates['output'] = 'mRuby2-A'

# Gate data by transfection marker expression
data = data.groupby('exp')[data.columns].apply(lambda x: base.gate_data(x,gates))
data.reset_index(inplace=True, drop=True)
max = 1e6
df = data[(data['expressing']) & (data['output']<max)]

In [ ]:
### Bin by marker quantiles
by = ['construct','exp','biorep']
df['bin_marker_quantiles'] = df.groupby(by)['marker'].transform(lambda x: pd.qcut(x, q=20, duplicates='drop'))
quantiles = df.groupby(by+['bin_marker_quantiles'])['marker'].median().rename('bin_marker_quantiles_median').reset_index()
df_quantiles = df.merge(quantiles, how='left', on=by+['bin_marker_quantiles'])


### Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp'])
stats = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats.columns = stats.columns.map(lambda i: base.rename_multilevel_cols(i))
stats['count'] = grouped['output'].count().reset_index()['output']
stats = stats.merge(metadata, how='left', on='construct')


### Quantile stats & slope
df_quantiles['bin_marker_quantiles_median_log'] = df_quantiles['bin_marker_quantiles_median'].apply(np.log10)
df_quantiles['output_log'] = df_quantiles['output'].apply(np.log10)

stat_list = [sp.stats.gmean, ]#sp.stats.gstd, sp.stats.variation]
grouped = df_quantiles.groupby(by=['construct','biorep','exp','bin_marker_quantiles_median'])
stats_quantiles = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats_quantiles.columns = stats_quantiles.columns.map(lambda i: base.rename_multilevel_cols(i))
stats_quantiles['count'] = grouped['output'].count().reset_index()['output']
stats_quantiles = stats_quantiles.merge(metadata, how='left', on='construct')
stats_quantiles['bin_marker_quantiles_median_log'] = stats_quantiles['bin_marker_quantiles_median'].apply(np.log10)
stats_quantiles['output_gmean_log'] = stats_quantiles['output_gmean'].apply(np.log10)


### Compute slope for all constructs
fits = stats_quantiles.groupby(['construct','exp','biorep'])[stats_quantiles.columns].apply(base.get_slope).reset_index()
fits['intercept'] = fits['intercept_log'].apply(lambda x: 10**x)
fits = fits.merge(metadata, how='left', on='construct')

In [ ]:
metadata['ts_label'] = metadata['ts_kind'].replace({'na': 'base', 'NT': 'unreg.', 'T': 'circuit'})

miR/TS characterization data (`data2`) from Emma

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'/'Emma'
exp11_path = base_path/'2022.10.11_EXP11'/'Data'
exp11_controls_path = base_path/'2022.10.11_EXP10'/'data_controls'
exp49_path = base_path/'2024.04.06_EXP11_replicates'/'Plate_1_EXP49'/'data_singlets'
exp50_path = base_path/'2024.04.06_EXP11_replicates'/'Plate_2_EXP50'/'data_singlets'
exp49_50_controls_path = base_path/'2024.04.06_EXP11_replicates'/'Plate_3_Controls'/'data_singlets'

plates = pd.DataFrame({
    'data_path': [base_path/'2022.10.04_EXP9'/'Data',
                  base_path/'2023.01.16_EXP12'/'Data',
                  base_path/'2023.02.09_EXP13'/'Data',
                  exp11_path, exp11_controls_path, 
                  exp49_path, exp50_path, 
                  exp49_50_controls_path, exp49_50_controls_path],
    
    'yaml_path': ([base_path/'2022.10.04_EXP9'/'Data'/'wells_KL.yaml']*3 + 
                  [exp11_path/'wells_KL.yaml', exp11_controls_path/'wells_KL.yaml', 
                   exp11_path/'wells_KL.yaml', exp11_path/'wells_KL.yaml', 
                   exp49_50_controls_path/'wells_KL.yaml', exp49_50_controls_path/'wells2_KL.yaml', ]),
    
    'biorep': [1,2,3,
               1,1,
               2,3,
               2,3],

    'exp': ['ELP_exp09', 'ELP_exp12', 'ELP_exp13',
            'ELP_exp11', 'ELP_exp11',
            'ELP_exp49', 'ELP_exp50',
            'ELP_exp49', 'ELP_exp50',],
})

cache_path = rd.rootdir/'output'/'fig_tuning-supp'/'data2.gzip'

# Load data
data2 = pd.DataFrame()
if cache_path.is_file(): data2 = pd.read_parquet(cache_path)
else: 
    channel_list = ['mCherry-A','mRuby2-A','FSC-A','SSC-A','tagBFP-A','mGL-A']
    data2 = rd.flow.load_groups_with_metadata(plates, columns=channel_list)

    # Remove negative channel values
    for c in channel_list: data2 = data2[data[c]>0]
    
    data2.dropna(inplace=True)
    data2.to_parquet(rd.outfile(cache_path))

# Add metadata for constructs
metadata_miR = pd.read_excel(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'miR-metadata.xlsx')
data2 = data2.merge(metadata_miR, how='left', on='miR_construct')
metadata_ts = pd.read_excel(rd.datadir/'projects'/'miR-iFFL'/'plasmids'/'ts-metadata.xlsx')
data2 = data2.merge(metadata_ts, how='left', on='ts_construct')
display(data2)

In [ ]:
# Gate cells
gates = pd.DataFrame()
channel_list = ['mGL-A', 'mRuby2-A']
for channel in channel_list:
    gates[channel] = data2[data2['ts_construct']=='UT'].groupby(['exp'])[channel].apply(lambda x: x.quantile(0.999))
gates.reset_index(inplace=True)

# Indicate which channels are relevant for each experiment
gates.sort_values(['exp'], inplace=True)
gates['marker'] = 'mRuby2-A'
gates['output'] = 'mGL-A'

# Gate data by transfection marker expression
data2 = data2.groupby('exp')[data2.columns].apply(lambda x: base.gate_data(x,gates))
data2.reset_index(inplace=True, drop=True)
max = 1e6
df2 = data2[(data2['expressing']) & (data2['output']>0)]

In [ ]:
data2['constructs'] = data2['miR_construct'] + '_' + data2['ts_construct']

In [ ]:
### Population stats
stat_list = [np.std, sp.stats.gmean, sp.stats.variation]
grouped = df2.groupby(by=['miR_construct','ts_construct','biorep','exp'])
stats2 = grouped[['marker','output']].agg(stat_list).reset_index().dropna()

# Rename columns as 'col_stat'
stats2.columns = stats2.columns.map(lambda i: base.rename_multilevel_cols(i))
stats2['count'] = grouped['output'].count().reset_index()['output']
stats2 = stats2.merge(metadata_miR, how='left', on='miR_construct')
stats2 = stats2.merge(metadata_ts, how='left', on='ts_construct')
stats2['constructs'] = stats2['miR_construct'] + '_' + stats2['ts_construct']

def get_fc(df):
    d = df.copy()
    baseline = d.loc[d['ts']=='none', 'output_gmean'].mean()
    d['output_gmean_fc'] = d['output_gmean'] / baseline
    return d

stats2 = stats2.groupby(by=['miR_construct','biorep','exp'])[stats2.columns].apply(get_fc).reset_index(drop=True)
fcs = stats2.groupby(by=['miR_construct','ts_construct',])[['output_gmean_fc']].apply('mean').reset_index()
fcs = fcs.merge(metadata_miR, how='left', on='miR_construct')
fcs = fcs.merge(metadata_ts, how='left', on='ts_construct')

'''
fcs_miRs = stats2[stats2['exp'].isin(['ELP_exp09', 'ELP_exp12', 'ELP_exp13'])].groupby(by=['miR_construct','ts_construct',])[['output_gmean_fc']].apply('mean').reset_index()
fcs_miRs = fcs_miRs.merge(metadata_miR, how='left', on='miR_construct')
fcs_miRs = fcs_miRs.merge(metadata_ts, how='left', on='ts_construct')
'''

# Identify subset of data with changing ts num
fcs['matched'] = fcs['miR'] == fcs['ts']
fcs.loc[fcs['ts']=='none', 'matched'] = True
stats2['matched'] = stats2['miR'] == stats2['ts']
stats2.loc[stats2['ts']=='none', 'matched'] = True

In [ ]:
# Create color palettes for miR/ts characterization

metadata_comb = data2.drop_duplicates(['constructs'])
metadata_comb['color'] = base.colors['gray']
metadata_comb['matched'] = metadata_comb['miR'] == metadata_comb['ts']
metadata_comb.loc[metadata_comb['matched'], 'color'] = base.colors['green']
metadata_comb.loc[metadata_comb['ts']=='none', 'color'] = 'black'

metadata_comb_dict = metadata_comb.set_index('constructs').to_dict('dict')
matched_palette = metadata_comb_dict['color']

metadata_comb['color'] = 'black'
metadata_comb.loc[metadata_comb['ts_num']>0, 'color'] = base.colors['green']
metadata_comb.loc[metadata_comb['ts_num']==2, 'color'] = metadata_comb.loc[metadata_comb['ts_num']==2, 'color'].apply(base.get_light_color)
metadata_comb.loc[metadata_comb['ts_num']==4, 'color'] = metadata_comb.loc[metadata_comb['ts_num']==4, 'color'].apply(base.get_light_color).apply(base.get_light_color)

metadata_comb_dict = metadata_comb.set_index('constructs').to_dict('dict')
ts_num_palette = metadata_comb_dict['color']

### Set up figure

In [ ]:
base_size = base.font_sizes['base_size']
smaller_size = base.font_sizes['smaller_size']

sns.set_style('ticks')
sns.set_context('paper')
sns.set_context('paper', font_scale=1.0, rc={'font.size': base_size, 'font.family': 'sans-serif', 'font.sans-serif':['Arial']})
plt.rcParams.update({'axes.titlesize': base_size, 'axes.labelsize': base_size, 'xtick.labelsize': smaller_size, 'ytick.labelsize': smaller_size,
                     'pdf.fonttype': 42, 'ytick.major.size': 3, 'xtick.major.size': 3})

In [ ]:
# Create the overall figure, gridspec, and add subfigure labels
fig = plt.figure(figsize=(6.8504,10))
fig_gridspec = matplotlib.gridspec.GridSpec(4, 6, figure=fig,
    wspace=0.4, hspace=0.4, height_ratios=[2,1.5,2,4.5], width_ratios=[1]*6)
subfigures = {
    'A': fig.add_subfigure(fig_gridspec[0,:2]),
    'B': fig.add_subfigure(fig_gridspec[0,2:4]),
    'C': fig.add_subfigure(fig_gridspec[1,:3]),
    'D': fig.add_subfigure(fig_gridspec[0,4:]),
    'E': fig.add_subfigure(fig_gridspec[1,3:]),
    'F': fig.add_subfigure(fig_gridspec[2,:3]),
    'G': fig.add_subfigure(fig_gridspec[2,3:]),
    'H': fig.add_subfigure(fig_gridspec[3,:2]),
    'I': fig.add_subfigure(fig_gridspec[3,2:]),
}
for label, subfig in subfigures.items():
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

fig_path = output_path/'fig_tuning-supp.pdf'
fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.5, top=0.2, right=0.2)
ax = subfig.subplots(1,1)

plot_df = fcs[(fcs['ts']!='na') & (fcs['miR']!='na') & (fcs['miR_promoter']=='hPGK.d') & (fcs['ts_num'].isin([0,2]))].pivot(index='miR', columns='ts', values='output_gmean_fc')
sns.heatmap(plot_df, annot=True, fmt='.2f', cmap=sns.light_palette(base.colors['green'], as_cmap=True), ax=ax)
ax.set(xlabel='target sites (x2)', ylabel='microRNA', title='Relative target expression')
for i in range(len(plot_df)-1):
    ax.add_patch(matplotlib.patches.Rectangle((i, i), 1, 1, fill=False, edgecolor='black', lw=1))

fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.45, bottom=0.625, top=0.2, right=0.2)
ax = subfig.subplots(1,1)

plot_df = fcs[(fcs['matched']==True) & (fcs['miR_promoter']=='hPGK.d') & (fcs['miR']!='none')].pivot(index='miR', columns='ts_num', values='output_gmean_fc')
sns.heatmap(plot_df, annot=True, fmt='.2f', cmap=sns.light_palette(base.colors['green'], as_cmap=True), ax=ax)
ax.set(xlabel='number of matched\ntarget sites', ylabel='microRNA', title='Relative target expression')

fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.5, top=0.3, right=0.1)
axes = subfig.subplots(1,5, sharey=True)

plot_df = stats2[(stats2['ts']!='na') & (stats2['miR']!='na') & (stats2['miR_promoter']=='hPGK.d') & (stats2['ts_num'].isin([0,2]))]

for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.stripplot(d, x='ts', y='output_gmean', ax=ax, s=4, edgecolor='white', linewidth=0.5,
                  hue='constructs', palette=matched_palette, legend=False)
    ax.set(yscale='log', xlabel='', title=f'{miR}')
    sns.despine(ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.minorticks_off()

axes[2].set(xlabel='target site (x2)')
ylabel_bbox = axes[0].get_yticklabels()[0]
title_bbox = axes[0].set_title(axes[0].get_title())
axes[0].annotate(text='miR:', xy=(1, 0.5), xycoords=(ylabel_bbox, title_bbox),
            ha="right", va="center", fontsize=base_size, )

        
fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.5, bottom=0.5, top=0.3, right=0.1)
axes = subfig.subplots(1,4, sharey=True)

plot_df = stats2[(stats2['matched']==True) & (stats2['miR_promoter']=='hPGK.d') & (stats2['miR']!='none')]

for ax, (miR, d) in zip(axes, plot_df.groupby('miR')): 
    sns.stripplot(d, x='ts_num', y='output_gmean', ax=ax, s=4, edgecolor='white', linewidth=0.5,
                  hue='constructs', palette=ts_num_palette, legend=False)
    ax.set(yscale='log', xlabel='', title=f'{miR}')
    sns.despine(ax=ax)
    ax.minorticks_off()

axes[1].set_xlabel('number of matched target sites', x=1, ha='center')#loc='left')
ylabel_bbox = axes[0].get_yticklabels()[0]
title_bbox = axes[0].set_title(axes[0].get_title())
axes[0].annotate(text='miR:', xy=(1, 0.5), xycoords=(ylabel_bbox, title_bbox),
            ha="right", va="center", fontsize=base_size, )

fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['F']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.6, top=0.6, right=0)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))

# EF1a miR controls design 1
xlim = (2e2,2e4)
ylim = (2e1,1e5)
miR_order = ['none', 'miR.FF5', 'miRE.FF5', 'miRE.FF4', 'miR.FF4',]
plot_df = stats[((((stats['group']=='miR')) & (stats['miR_loc']=='CDS')) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
plot_df2 = fits[((((fits['group']=='miR')) & (fits['miR_loc']=='CDS')) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['miR'] = plot_df2['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['construct'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='miR', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['construct'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0','','','','1'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(fig_path), bbox_inches='tight')

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.4, bottom=0.6, top=0.6, right=0)
axes = subfig.subplots(1,3, gridspec_kw=dict(width_ratios=(1,1,1), wspace=0.2))

# EF1a miR controls design 1
xlim = (2e2,2e4)
ylim = (2e1,1e5)
ts_order = ['none','FF6x1','FF5x1','FF4x1','FF3x1']
plot_df = stats[((((stats['group']=='ts3')) & (stats['ts_num']==1)) | (stats['group']=='base')) &
                (stats['promoter']=='EF1a')].copy()
plot_df['ts'] = plot_df['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))
plot_df2 = fits[((((fits['group']=='ts3')) & (fits['ts_num']==1)) | (fits['group']=='base')) &
                (fits['promoter']=='EF1a')].copy()
plot_df2['ts'] = plot_df2['ts'].astype(pd.api.types.CategoricalDtype(categories=ts_order, ordered=True))

# stat gmean
ax = axes[0]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_gmean', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Geometric mean', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# stat std
ax = axes[1]
for construct, group in plot_df.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='output_std', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')
sns.despine(ax=ax)
ax.set(title='Standard deviation', xlim=(-0.5, len(plot_df['ts'].unique())-0.5), xlabel='', ylim=ylim, ylabel='', yscale='log',
       yticklabels=['']*(len(ax.get_yticklabels())-1))
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_std'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

# slope
ax = axes[2]
for construct, group in plot_df2.groupby('construct'):
    sns.stripplot(data=group, x='ts', y='slope', hue='construct', palette=main_palette,
                    legend=False, ax=ax, marker=main_markers[construct], s=4, jitter=0.2, linewidth=0.5, edgecolor='white')

ax.set(title='Slope', xlim=(-0.5, len(plot_df2['ts'].unique())-0.5), xlabel='', ylim=(0,1.2), ylabel='',
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0','','','','1'])
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
sns.despine(ax=ax)
marker_baseline = fits.loc[(fits['group']=='marker'), 'slope'].mean()
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(fig_path), bbox_inches='tight')